In [2]:
import MetaTrader5 as mt5
import pandas as pd

In [3]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

In [6]:
ativo = 'EURUSD'
df1 = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M15, 0,99999))
df1['data'] = pd.to_datetime(df1['time'], unit='s')
df1 = df1[df1['data'] > '2022-01-01']
df1

,time,open,high,low,close,tick_volume,spread,real_volume,data
75140,1641168000,1.13709,1.13730,1.13654,1.13719,121,10,0,2022-01-03 00:00:00
75141,1641168900,1.13721,1.13723,1.13701,1.13719,95,29,0,2022-01-03 00:15:00
75142,1641169800,1.13673,1.13719,1.13673,1.13719,21,66,0,2022-01-03 00:30:00
75143,1641170700,1.13719,1.13724,1.13705,1.13707,31,31,0,2022-01-03 00:45:00
75144,1641171600,1.13707,1.13781,1.13692,1.13724,1176,11,0,2022-01-03 01:00:00
...,...,...,...,...,...,...,...,...,...
99994,1672440300,1.07018,1.07077,1.07014,1.07049,1015,11,0,2022-12-30 22:45:00
99995,1672441200,1.07050,1.07068,1.07016,1.07029,531,11,0,2022-12-30 23:00:00
99996,1672442100,1.07030,1.07041,1.06995,1.07036,439,13,0,2022-12-30 23:15:00
99997,1672443000,1.07036,1.07038,1.07005,1.07020,348,11,0,2022-12-30 23:30:00


In [18]:
ativo = 'WIN@N'
df1 = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M10, 0,13700))
df1['data'] = pd.to_datetime(df1['time'], unit='s')
df1

KeyError: 'time'

In [11]:
df = df1[['data','open','high','low','close','real_volume']]
df = df.rename(columns = {'real_volume':'vol'})

In [12]:
df.to_csv('EURUSD15_new.csv', index = False)

In [65]:
df1

""


#### Verificar posições abertas

In [30]:
import time

In [31]:

while True:
    #colonnes = ["ticket", "position", "symbol", "volume", "magic", "profit", "pip_diff", "tp", "sl","trade_size"]
    # Pega as posições em aberto
    liste = mt5.positions_get()
    for element in liste:
        '''element_pandas = pd.DataFrame([element.ticket, element.type, element.symbol, element.volume, element.magic,
                                        element.profit, element.price_current - element.price_open, element.tp,
                                        element.sl, mt5.symbol_info(element.symbol).trade_contract_size],
                                        index=colonnes).transpose()'''

        pip_diff = abs(element.sl - element.price_current)
        symbol = element.symbol
        position = element.type
        vol = element.volume
        price = element.price_open
        sl = element.sl
        ticket = element.ticket

        if pip_diff > 0.0005 * 2:

            if position == 0:
                request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "symbol": symbol,
                "position": ticket,
                "volume": vol,
                "type": mt5.ORDER_TYPE_BUY,
                "price": price,
                "sl": sl + 0.0005,
                "type_filling": mt5.ORDER_FILLING_IOC,
                "type_time": mt5.ORDER_TIME_GTC,
                }
                information = mt5.order_send(request)
                print(information)

            else:
                request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "symbol": symbol,
                "position": ticket,
                "volume": vol,
                "type": mt5.ORDER_TYPE_SELL,
                "price": price,
                "sl": sl - 0.0005,
                "type_filling": mt5.ORDER_FILLING_IOC,
                "type_time": mt5.ORDER_TIME_GTC,
                }
                information = mt5.order_send(request)
                print(information)

        #summary = pd.concat((summary, element_pandas), axis=0)

    time.sleep(2)

OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=11, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUSD', volume=1.0, price=1.08584, stoplimit=0.0, sl=1.08556, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='', position=1566045347, position_by=0))
OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=13, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUSD', volume=1.0, price=1.08584, stoplimit=0.0, sl=1.08614, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='', position=1566045347, position_by=0))
OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=14, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUS

KeyboardInterrupt: 

In [22]:
mt5.symbol_info('GBPUSD').filling_mode

1

In [10]:
summary

,ticket,position,symbol,volume,magic,profit,pip_diff,tp,sl,trade_size
0,1566045347,1,EURUSD,1.0,0,15.0,-0.00015,0.0,0.0,100000.0


In [11]:
mt5.positions_get()[0]

TradePosition(ticket=1566045347, time=1673556364, time_msc=1673556364307, time_update=1673556364, time_update_msc=1673556364307, type=1, magic=0, identifier=1566045347, reason=0, volume=1.0, price_open=1.08584, sl=0.0, tp=0.0, price_current=1.08578, swap=0.0, profit=6.0, symbol='EURUSD', comment='', external_id='')

In [17]:
information

In [29]:
request = {
"action": mt5.TRADE_ACTION_SLTP,
"symbol": symbol,
"position": ticket,
"volume": vol,
"type": mt5.ORDER_TYPE_SELL,
"price": price,
"sl": 1.08795,
"type_filling": mt5.ORDER_FILLING_IOC,
"type_time": mt5.ORDER_TIME_GTC,
}

information = mt5.order_send(request)
print(information)

OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=6, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUSD', volume=1.0, price=1.08584, stoplimit=0.0, sl=1.08795, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='', position=1566045347, position_by=0))


In [25]:
vol

1.0